# Finding/opening ACCESS-OM3 `MC_25km_jra_ryf-1.0-beta` test run output using intake

This notebook demonstrates how to using intake-esm to find and load data data from the ACCESS-OM3 `MC_25km_jra_ryf-1.0-beta` test run.

For more information about using intake-esm to find and load data, see:
- the [intake-esm documentation](https://intake-esm.readthedocs.io/en/stable/)
- [this section](https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/quickstart.html#using-an-intake-esm-datastore) of the access-nri-intake-catalog documentation

In [1]:
#This cell must be in all notebooks!
#It allows us to run all the notebooks at once, this cell has a tag "parameters" which allows us to pass in 
# arguments externally using papermill (see mkfigs.sh for details)

### USER EDIT start
esm_file = "/scratch/ps29/nd0349/access-om3/archive/IC4M8-MCW-100km_jra_ryf/experiment_datastore.json"
dpi=300
### USER EDIT stop

import os
from matplotlib import rcParams
%matplotlib inline
rcParams["figure.dpi"]= dpi

plotfolder=f"/g/data/{os.environ['PROJECT']}/{os.environ['USER']}/access-om3-paper-figs/"
os.makedirs(plotfolder, exist_ok=True)

 # a similar cell under this means it's being run in batch
print("ESM datastore path: ",esm_file)
print("Plot folder path: ",plotfolder)

ESM datastore path:  /scratch/ps29/nd0349/access-om3/archive/IC4M8-MCW-100km_jra_ryf/experiment_datastore.json
Plot folder path:  /g/data/ps29/nd0349/access-om3-paper-figs/


In [12]:
import xarray as xr
import cf_xarray
import intake
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from distributed import Client
# Import my functions
functions_path = os.path.abspath("/home/566/nd0349/access-om3-analysis/functions")
if functions_path not in sys.path:
    sys.path.append(functions_path)
from get_files import *
from plot_settings import *
from fstd import *
from attenuation_models import *
test()

Functions from get_files.py imported!


In [3]:
client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 14
Total threads: 14,Total memory: 63.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40623,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:40555,Total threads: 1
Dashboard: /proxy/37211/status,Memory: 4.50 GiB
Nanny: tcp://127.0.0.1:46465,


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-688463' coro=<Client._gather.<locals>.wait() done, defined at /g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/distributed/client.py:2377> exception=AllExit()>
Traceback (most recent call last):
  File "/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/distributed/client.py", line 2386, in wait
    raise AllExit()
distributed.client.AllExit


In [4]:
print(client.dashboard_link)

/proxy/8787/status


### Open the intake-esm datastore

In [5]:
COLUMNS_WITH_ITERABLES = [
        "variable",
        "variable_long_name",
        "variable_standard_name",
        "variable_cell_methods",
        "variable_units"
]

datastore = intake.open_esm_datastore(
    esm_file,
    columns_with_iterables=COLUMNS_WITH_ITERABLES
)


### What ocean variables are available at monthly frequency?

In [6]:
def available_variables(datastore):
    """Return a pandas dataframe summarising the variables in a datastore"""
    variable_columns = [col for col in datastore.df.columns if "variable" in col]
    return (
        datastore.df[variable_columns]
        .explode(variable_columns)
        .drop_duplicates()
        .set_index("variable")
        .sort_index()
    )

In [8]:
datastore_filtered = datastore.search(realm="seaIce", frequency="1mon")

available_variables(datastore_filtered)

,variable_long_name,variable_standard_name,variable_cell_methods,variable_units
variable,,,,
ANGLE,angle grid makes with latitude line on U grid,,,radians
ANGLET,angle grid makes with latitude line on T grid,,,radians
NCAT,category maximum thickness,,,m
Tair_m,air temperature,,time: mean,C
Tsfc_m,snow/ice surface temperature,,time: mean,C
...,...,...,...,...
vatm_m,atm velocity (y),,time: mean,m/s
vicen_m,"ice volume, categories",,time: mean,m
vort_m,strain rate (vorticity),,time: mean,%/day


### Load monthly snow thickness (`hs_m`) and plot the field at the last available time

In [22]:
# ds_hs = datastore.search(variable="hs_m", frequency="1mon").to_dask(
#     xarray_combine_by_coords_kwargs = dict( # These kwargs can make things faster
#         compat="override",
#         data_vars="minimal",
#         coords="minimal",
#     ),
#     xarray_open_kwargs = dict(
#         # chunks={"yh": -1, "xh": -1}, # Good for spatial operations, but not temporal
#         decode_timedelta=True
#     )
# )

In [25]:
files[-10:]

[PosixPath('/g/data/ps29/nd0349/ia40/waves-025/history/iceh.2014-12-22.nc'),
 PosixPath('/g/data/ps29/nd0349/ia40/waves-025/history/iceh.2014-12-23.nc'),
 PosixPath('/g/data/ps29/nd0349/ia40/waves-025/history/iceh.2014-12-24.nc'),
 PosixPath('/g/data/ps29/nd0349/ia40/waves-025/history/iceh.2014-12-25.nc'),
 PosixPath('/g/data/ps29/nd0349/ia40/waves-025/history/iceh.2014-12-26.nc'),
 PosixPath('/g/data/ps29/nd0349/ia40/waves-025/history/iceh.2014-12-27.nc'),
 PosixPath('/g/data/ps29/nd0349/ia40/waves-025/history/iceh.2014-12-28.nc'),
 PosixPath('/g/data/ps29/nd0349/ia40/waves-025/history/iceh.2014-12-29.nc'),
 PosixPath('/g/data/ps29/nd0349/ia40/waves-025/history/iceh.2014-12-30.nc'),
 PosixPath('/g/data/ps29/nd0349/ia40/waves-025/history/iceh.2014-12-31.nc')]

In [28]:
from pathlib import Path
import xarray as xr

files = sorted(Path("/g/data/ps29/nd0349/ia40/waves-025").glob("history/iceh.*.nc"))
ds_hs = xr.open_mfdataset(files[-180], combine='by_coords')
ds_hs

<xarray.Dataset> Size: 496MB
Dimensions:        (time: 1, nj: 324, ni: 1440, nc: 5, nf: 12, nkaer: 5, d2: 2)
Coordinates:
    NCAT           (nc) float32 20B dask.array<chunksize=(5,), meta=np.ndarray>
    NFSD           (nf) float32 48B dask.array<chunksize=(12,), meta=np.ndarray>
    TLAT           (nj, ni) float32 2MB dask.array<chunksize=(324, 1440), meta=np.ndarray>
    TLON           (nj, ni) float32 2MB dask.array<chunksize=(324, 1440), meta=np.ndarray>
    ULAT           (nj, ni) float32 2MB dask.array<chunksize=(324, 1440), meta=np.ndarray>
    ULON           (nj, ni) float32 2MB dask.array<chunksize=(324, 1440), meta=np.ndarray>
  * time           (time) datetime64[ns] 8B 2014-07-06
Dimensions without coordinates: nj, ni, nc, nf, nkaer, d2
Data variables: (12/107)
    FYarea         (time, nj, ni) float32 2MB dask.array<chunksize=(1, 324, 1440), meta=np.ndarray>
    HTE            (nj, ni) float32 2MB dask.array<chunksize=(324, 1440), meta=np.ndarray>
    HTN            (nj, ni) float32 2MB dask.array<chunksize=(324, 1440), meta=np.ndarray>
    Tair           (time, nj, ni) float32 2MB dask.array<chunksize=(1, 324, 1440), meta=np.ndarray>
    Tsfc           (time, nj, ni) float32 2MB dask.array<chunksize=(1, 324, 1440), meta=np.ndarray>
    VGRDa          (nkaer) float32 20B dask.array<chunksize=(5,), meta=np.ndarray>
    ...             ...
    vraftn         (time, nc, nj, ni) float32 9MB dask.array<chunksize=(1, 5, 324, 1440), meta=np.ndarray>
    vrdg           (time, nj, ni) float32 2MB dask.array<chunksize=(1, 324, 1440), meta=np.ndarray>
    vredistn       (time, nc, nj, ni) float32 9MB dask.array<chunksize=(1, 5, 324, 1440), meta=np.ndarray>
    vsnon          (time, nc, nj, ni) float32 9MB dask.array<chunksize=(1, 5, 324, 1440), meta=np.ndarray>
    vvel           (time, nj, ni) float32 2MB dask.array<chunksize=(1, 324, 1440), meta=np.ndarray>
    wave_sig_ht    (time, nj, ni) float32 2MB dask.array<chunksize=(1, 324, 1440), meta=np.ndarray>
Attributes:
    title:        sea ice model output for CICE
    contents:     Diagnostic and Prognostic Variables
    source:       Los Alamos Sea Ice Model, CICE_6.2.0
    comment:      This year has 365 days
    comment2:     File written on model date 20140706
    comment3:     seconds elapsed into model date:      0
    conventions:  CF-1.0
    history:      Sun Dec 17 16:03:26 2023: ncks -d nj,0,323 -O /g/data/ia40/...
    io_flavor:    io_netcdf
    NCO:          netCDF Operators version 5.0.7 (Homepage = http://nco.sf.ne...

In [ ]:
time_idx = -1
hemisphere = "south"
dims = [1, 1]
fig, axes, projection = basic_axis(dims, hemisphere=hemisphere)

# ds_hs["hs_m"].isel(time=time_idx).plot(ax=axes[0])
ds_hs["hs"].isel(time=time_idx).plot(ax=axes[0])
axes[0].coastlines()
_ = axes[0].gridlines()

#### Fixing the plot axes

Notice that the white land-masked regions are distorted away from the coastlines in the Arctic in the above plot. This is because a tripolar grid is used, so the grid lines are not zonal and meridional north of 65N, and consequently the nominal 1D coordinates `xh` and `yh` are incorrect. To fix this we need to use 2d coordinates `geolon` and `geolat`.

We can get these coorindates from the intake-esm datastore. However, note that `geolon` and `geolat` contain NaNs in regions where processors were masked over land. Below we replace these NaNs with zeros so that the coordinates can be used for plotting.

In [9]:
coords = datastore.search(variable=["geolat", "geolon"]).to_dask().compute()
coords = coords.fillna(0.0)

zos = zos.assign_coords(coords)

/g/data/xp65/public/apps/med_conda/envs/analysis3-25.09/lib/python3.11/site-packages/intake_esm/source.py:308: ConcatenationWarning: Attempting to concatenate datasets without valid dimension coordinates: retaining only first dataset. Request valid dimension coordinate to silence this warning.
  warnings.warn(


In [ ]:
proj = ccrs.PlateCarree()
fig, ax = plt.subplots(figsize=(15,6), subplot_kw=dict(projection=proj))

zos["zos"].isel(time=-1).plot(ax=ax, x="geolon", y="geolat")

ax.coastlines()
_ = ax.gridlines()
plt.savefig(plotfolder+'exampleout.png')

In [ ]:
client.close()